In [2]:
train_size = 47500

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [4]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [5]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [6]:
XTX = np.concatenate((X, TX), axis=0)

In [7]:
X = None
TX = None

In [8]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [9]:
XTX_square = None
XTX_cumsum = None
XTX = None

In [10]:
XTX_all.shape

(50000, 30000)

In [11]:
X, X_test = XTX_all[:train_size], XTX_all[train_size:]
print(X.shape, X_test.shape)

(47500, 30000) (2500, 30000)


In [12]:
# calculate type 1 error
def err1(y_pred, y_true):
    return 't1e', np.sum(1.0 * np.abs(y_pred - y_true.get_label())) / len(y_pred)

# calculate type 2 errr
def err2(y_pred, y_true):
    return 't2e', np.sum(np.abs(y_pred - y_true.get_label()) / y_true.get_label()) / len(y_pred)
scorer = {'t1':err1, 't2':err2}

In [13]:
params = {'booster':'gbtree', 'learning_rate':0.05, 'max_depth':5, 'n_estimators':500, 'tree_method':'hist', 'n_jobs':16}

In [ ]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t1'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t1_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t1_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature_imp.csv', index=False, header=False)

[0]	validation_0-rmse:0.282786	validation_0-t1e:0.245695
Multiple eval metrics have been passed: 'validation_0-t1e' will be used for early stopping.

Will train until validation_0-t1e hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.274739	validation_0-t1e:0.238883
[2]	validation_0-rmse:0.267177	validation_0-t1e:0.232454
[3]	validation_0-rmse:0.260161	validation_0-t1e:0.226468
[4]	validation_0-rmse:0.253332	validation_0-t1e:0.220569
[5]	validation_0-rmse:0.247061	validation_0-t1e:0.215193
[6]	validation_0-rmse:0.241096	validation_0-t1e:0.21006
[7]	validation_0-rmse:0.235528	validation_0-t1e:0.205191
[8]	validation_0-rmse:0.230447	validation_0-t1e:0.200714
[9]	validation_0-rmse:0.225574	validation_0-t1e:0.196391
[10]	validation_0-rmse:0.220812	validation_0-t1e:0.192147
[11]	validation_0-rmse:0.21651	validation_0-t1e:0.18829
[12]	validation_0-rmse:0.212407	validation_0-t1e:0.184587
[13]	validation_0-rmse:0.208641	validation_0-t1e:0.181132
[14]	validation_0-rmse:0.204993	validation_0-

[139]	validation_0-rmse:0.135585	validation_0-t1e:0.102981
[140]	validation_0-rmse:0.135533	validation_0-t1e:0.102932
[141]	validation_0-rmse:0.135474	validation_0-t1e:0.102859
[142]	validation_0-rmse:0.135426	validation_0-t1e:0.102797
[143]	validation_0-rmse:0.135379	validation_0-t1e:0.102754
[144]	validation_0-rmse:0.135376	validation_0-t1e:0.102748
[145]	validation_0-rmse:0.135353	validation_0-t1e:0.102727
[146]	validation_0-rmse:0.135271	validation_0-t1e:0.102624
[147]	validation_0-rmse:0.135259	validation_0-t1e:0.1026
[148]	validation_0-rmse:0.13518	validation_0-t1e:0.102529
[149]	validation_0-rmse:0.135164	validation_0-t1e:0.10252
[150]	validation_0-rmse:0.135161	validation_0-t1e:0.102504
[151]	validation_0-rmse:0.135115	validation_0-t1e:0.102448
[152]	validation_0-rmse:0.135098	validation_0-t1e:0.102433
[153]	validation_0-rmse:0.135075	validation_0-t1e:0.102416
[154]	validation_0-rmse:0.135032	validation_0-t1e:0.102372
[155]	validation_0-rmse:0.134941	validation_0-t1e:0.102285
[

[279]	validation_0-rmse:0.133148	validation_0-t1e:0.100348
[280]	validation_0-rmse:0.13315	validation_0-t1e:0.100353
[281]	validation_0-rmse:0.133154	validation_0-t1e:0.100348
[282]	validation_0-rmse:0.133146	validation_0-t1e:0.100341
Stopping. Best iteration:
[277]	validation_0-rmse:0.133139	validation_0-t1e:0.10034

[0]	validation_0-rmse:0.27976	validation_0-t1e:0.242752
Multiple eval metrics have been passed: 'validation_0-t1e' will be used for early stopping.

Will train until validation_0-t1e hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.27175	validation_0-t1e:0.235999
[2]	validation_0-rmse:0.264222	validation_0-t1e:0.229613
[3]	validation_0-rmse:0.257221	validation_0-t1e:0.223662
[4]	validation_0-rmse:0.25061	validation_0-t1e:0.218012
[5]	validation_0-rmse:0.244427	validation_0-t1e:0.21272
[6]	validation_0-rmse:0.238492	validation_0-t1e:0.207625
[7]	validation_0-rmse:0.232948	validation_0-t1e:0.202775
[8]	validation_0-rmse:0.227837	validation_0-t1e:0.198309
[9]	validation_

[134]	validation_0-rmse:0.135864	validation_0-t1e:0.103383
[135]	validation_0-rmse:0.135806	validation_0-t1e:0.103299
[136]	validation_0-rmse:0.135747	validation_0-t1e:0.103259
[137]	validation_0-rmse:0.135693	validation_0-t1e:0.10321
[138]	validation_0-rmse:0.135653	validation_0-t1e:0.103172
[139]	validation_0-rmse:0.135609	validation_0-t1e:0.103124
[140]	validation_0-rmse:0.135601	validation_0-t1e:0.103125
[141]	validation_0-rmse:0.135508	validation_0-t1e:0.103011
[142]	validation_0-rmse:0.135512	validation_0-t1e:0.103015
[143]	validation_0-rmse:0.135465	validation_0-t1e:0.102961
[144]	validation_0-rmse:0.135422	validation_0-t1e:0.10292
[145]	validation_0-rmse:0.135393	validation_0-t1e:0.102888
[146]	validation_0-rmse:0.135329	validation_0-t1e:0.102834
[147]	validation_0-rmse:0.135306	validation_0-t1e:0.102797
[148]	validation_0-rmse:0.135234	validation_0-t1e:0.102723
[149]	validation_0-rmse:0.135195	validation_0-t1e:0.102673
[150]	validation_0-rmse:0.135155	validation_0-t1e:0.102616

[274]	validation_0-rmse:0.133398	validation_0-t1e:0.100659
[275]	validation_0-rmse:0.133397	validation_0-t1e:0.100658
[276]	validation_0-rmse:0.13339	validation_0-t1e:0.100653
[277]	validation_0-rmse:0.133356	validation_0-t1e:0.100621
[278]	validation_0-rmse:0.133347	validation_0-t1e:0.100621
[279]	validation_0-rmse:0.133328	validation_0-t1e:0.100601
[280]	validation_0-rmse:0.133318	validation_0-t1e:0.100587
[281]	validation_0-rmse:0.133302	validation_0-t1e:0.100579
[282]	validation_0-rmse:0.133292	validation_0-t1e:0.100571
[283]	validation_0-rmse:0.133287	validation_0-t1e:0.100564
[284]	validation_0-rmse:0.133288	validation_0-t1e:0.100564
[285]	validation_0-rmse:0.133265	validation_0-t1e:0.100528
[286]	validation_0-rmse:0.133271	validation_0-t1e:0.100523
[287]	validation_0-rmse:0.133269	validation_0-t1e:0.100505
[288]	validation_0-rmse:0.133261	validation_0-t1e:0.100492
[289]	validation_0-rmse:0.133261	validation_0-t1e:0.100496
[290]	validation_0-rmse:0.133222	validation_0-t1e:0.10045

[91]	validation_0-rmse:0.139603	validation_0-t1e:0.107411
[92]	validation_0-rmse:0.139487	validation_0-t1e:0.107263
[93]	validation_0-rmse:0.139333	validation_0-t1e:0.107091
[94]	validation_0-rmse:0.139192	validation_0-t1e:0.106936
[95]	validation_0-rmse:0.139026	validation_0-t1e:0.106752
[96]	validation_0-rmse:0.138832	validation_0-t1e:0.106528
[97]	validation_0-rmse:0.138762	validation_0-t1e:0.106448
[98]	validation_0-rmse:0.138643	validation_0-t1e:0.106311
[99]	validation_0-rmse:0.13857	validation_0-t1e:0.106246
[100]	validation_0-rmse:0.138494	validation_0-t1e:0.106169
[101]	validation_0-rmse:0.138371	validation_0-t1e:0.106047
[102]	validation_0-rmse:0.138282	validation_0-t1e:0.105945
[103]	validation_0-rmse:0.138196	validation_0-t1e:0.105861
[104]	validation_0-rmse:0.138083	validation_0-t1e:0.105721
[105]	validation_0-rmse:0.13798	validation_0-t1e:0.105611
[106]	validation_0-rmse:0.137867	validation_0-t1e:0.105459
[107]	validation_0-rmse:0.137824	validation_0-t1e:0.105407
[108]	va

[9]	validation_0-rmse:0.223487	validation_0-t1e:0.194737
[10]	validation_0-rmse:0.21916	validation_0-t1e:0.190855
[11]	validation_0-rmse:0.214778	validation_0-t1e:0.186924
[12]	validation_0-rmse:0.210797	validation_0-t1e:0.183323
[13]	validation_0-rmse:0.207278	validation_0-t1e:0.180095
[14]	validation_0-rmse:0.203754	validation_0-t1e:0.176895
[15]	validation_0-rmse:0.200413	validation_0-t1e:0.173756
[16]	validation_0-rmse:0.197469	validation_0-t1e:0.170965
[17]	validation_0-rmse:0.194361	validation_0-t1e:0.167969
[18]	validation_0-rmse:0.191766	validation_0-t1e:0.165483
[19]	validation_0-rmse:0.189073	validation_0-t1e:0.162836
[20]	validation_0-rmse:0.186886	validation_0-t1e:0.160671
[21]	validation_0-rmse:0.184737	validation_0-t1e:0.158525
[22]	validation_0-rmse:0.182623	validation_0-t1e:0.156379
[23]	validation_0-rmse:0.180433	validation_0-t1e:0.154153
[24]	validation_0-rmse:0.178411	validation_0-t1e:0.152091
[25]	validation_0-rmse:0.176727	validation_0-t1e:0.150337
[26]	validation_

In [14]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t2'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t2_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t2_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature2_imp.csv', index=False, header=False)

[03:42:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.282786	validation_0-t2e:3.64539
Multiple eval metrics have been passed: 'validation_0-t2e' will be used for early stopping.

Will train until validation_0-t2e hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.274739	validation_0-t2e:3.52897
[2]	validation_0-rmse:0.267177	validation_0-t2e:3.41601
[3]	validation_0-rmse:0.260161	validation_0-t2e:3.31258
[4]	validation_0-rmse:0.253332	validation_0-t2e:3.21174
[5]	validation_0-rmse:0.247061	validation_0-t2e:3.11733
[6]	validation_0-rmse:0.241096	validation_0-t2e:3.02252
[7]	validation_0-rmse:0.235528	validation_0-t2e:2.93238
[8]	validation_0-rmse:0.230447	validation_0-t2e:2.84947
[9]	validation_0-rmse:0.225574	validation_0-t2e:2.7626
[10]	validation_0-rmse:0.220812	validation_0-t2e:2.68712
[11]	validation_0-rmse:0.216511	validation_0-t2e:2.6136
[12]	validation_0-rmse:0.212407	validation_0-

[138]	validation_0-rmse:0.135609	validation_0-t2e:0.799215
[139]	validation_0-rmse:0.135585	validation_0-t2e:0.796975
[140]	validation_0-rmse:0.135533	validation_0-t2e:0.79668
[141]	validation_0-rmse:0.135474	validation_0-t2e:0.794788
[142]	validation_0-rmse:0.135426	validation_0-t2e:0.794257
[143]	validation_0-rmse:0.135379	validation_0-t2e:0.793933
[144]	validation_0-rmse:0.135376	validation_0-t2e:0.793824
[145]	validation_0-rmse:0.135353	validation_0-t2e:0.793782
[146]	validation_0-rmse:0.135271	validation_0-t2e:0.790383
[147]	validation_0-rmse:0.135259	validation_0-t2e:0.789181
[148]	validation_0-rmse:0.13518	validation_0-t2e:0.788617
[149]	validation_0-rmse:0.135164	validation_0-t2e:0.788161
[150]	validation_0-rmse:0.135161	validation_0-t2e:0.786339
[151]	validation_0-rmse:0.135115	validation_0-t2e:0.784881
[152]	validation_0-rmse:0.135098	validation_0-t2e:0.785412
[153]	validation_0-rmse:0.135075	validation_0-t2e:0.78549
[154]	validation_0-rmse:0.135032	validation_0-t2e:0.785134


[89]	validation_0-rmse:0.139666	validation_0-t2e:0.83137
[90]	validation_0-rmse:0.139495	validation_0-t2e:0.828883
[91]	validation_0-rmse:0.139364	validation_0-t2e:0.828093
[92]	validation_0-rmse:0.139192	validation_0-t2e:0.823456
[93]	validation_0-rmse:0.139056	validation_0-t2e:0.815927
[94]	validation_0-rmse:0.138941	validation_0-t2e:0.814419
[95]	validation_0-rmse:0.138831	validation_0-t2e:0.813367
[96]	validation_0-rmse:0.138715	validation_0-t2e:0.810644
[97]	validation_0-rmse:0.13861	validation_0-t2e:0.809278
[98]	validation_0-rmse:0.138499	validation_0-t2e:0.807046
[99]	validation_0-rmse:0.138417	validation_0-t2e:0.806653
[100]	validation_0-rmse:0.138307	validation_0-t2e:0.805078
[101]	validation_0-rmse:0.138228	validation_0-t2e:0.80253
[102]	validation_0-rmse:0.138147	validation_0-t2e:0.800737
[103]	validation_0-rmse:0.138034	validation_0-t2e:0.798959
[104]	validation_0-rmse:0.137938	validation_0-t2e:0.797688
[105]	validation_0-rmse:0.137873	validation_0-t2e:0.797264
[106]	valid

[86]	validation_0-rmse:0.140378	validation_0-t2e:0.692976
[87]	validation_0-rmse:0.14014	validation_0-t2e:0.692378
[88]	validation_0-rmse:0.140012	validation_0-t2e:0.696072
[89]	validation_0-rmse:0.139878	validation_0-t2e:0.694791
[90]	validation_0-rmse:0.139727	validation_0-t2e:0.691704
[91]	validation_0-rmse:0.139603	validation_0-t2e:0.691092
[92]	validation_0-rmse:0.139487	validation_0-t2e:0.69039
[93]	validation_0-rmse:0.139333	validation_0-t2e:0.686734
[94]	validation_0-rmse:0.139192	validation_0-t2e:0.68968
[95]	validation_0-rmse:0.139026	validation_0-t2e:0.68988
[96]	validation_0-rmse:0.138832	validation_0-t2e:0.690112
[97]	validation_0-rmse:0.138762	validation_0-t2e:0.690694
[98]	validation_0-rmse:0.138643	validation_0-t2e:0.687008
Stopping. Best iteration:
[93]	validation_0-rmse:0.139333	validation_0-t2e:0.686734

[06:42:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.279987	validat

[124]	validation_0-rmse:0.137193	validation_0-t2e:0.750958
[125]	validation_0-rmse:0.137146	validation_0-t2e:0.748738
[126]	validation_0-rmse:0.137099	validation_0-t2e:0.748891
[127]	validation_0-rmse:0.137013	validation_0-t2e:0.748936
[128]	validation_0-rmse:0.136936	validation_0-t2e:0.747858
[129]	validation_0-rmse:0.136872	validation_0-t2e:0.747426
[130]	validation_0-rmse:0.13682	validation_0-t2e:0.749509
[131]	validation_0-rmse:0.136797	validation_0-t2e:0.749249
[132]	validation_0-rmse:0.136751	validation_0-t2e:0.748317
[133]	validation_0-rmse:0.136702	validation_0-t2e:0.747717
[134]	validation_0-rmse:0.136652	validation_0-t2e:0.746683
[135]	validation_0-rmse:0.136573	validation_0-t2e:0.748153
[136]	validation_0-rmse:0.136514	validation_0-t2e:0.745271
[137]	validation_0-rmse:0.136451	validation_0-t2e:0.744792
[138]	validation_0-rmse:0.136446	validation_0-t2e:0.742802
[139]	validation_0-rmse:0.136429	validation_0-t2e:0.742491
[140]	validation_0-rmse:0.136418	validation_0-t2e:0.74244

...

[168]	validation_0-rmse:0.00927	validation_0-t2e:0.009773
[169]	validation_0-rmse:0.009269	validation_0-t2e:0.009773
[170]	validation_0-rmse:0.009266	validation_0-t2e:0.00977
[171]	validation_0-rmse:0.009265	validation_0-t2e:0.009769
[172]	validation_0-rmse:0.009264	validation_0-t2e:0.009768
[173]	validation_0-rmse:0.009263	validation_0-t2e:0.009767
[174]	validation_0-rmse:0.009262	validation_0-t2e:0.009766
[175]	validation_0-rmse:0.009263	validation_0-t2e:0.009767
[176]	validation_0-rmse:0.009261	validation_0-t2e:0.009765
[177]	validation_0-rmse:0.009258	validation_0-t2e:0.009762
[178]	validation_0-rmse:0.009257	validation_0-t2e:0.009761
[179]	validation_0-rmse:0.009255	validation_0-t2e:0.009758
[180]	validation_0-rmse:0.009255	validation_0-t2e:0.009757
[181]	validation_0-rmse:0.009253	validation_0-t2e:0.009754
[182]	validation_0-rmse:0.009253	validation_0-t2e:0.009754
[183]	validation_0-rmse:0.009252	validation_0-t2e:0.009753
[184]	validation_0-rmse:0.009252	validation_0-t2e:0.009753

[7]	validation_0-rmse:0.190913	validation_0-t2e:0.202966
[8]	validation_0-rmse:0.181429	validation_0-t2e:0.192832
[9]	validation_0-rmse:0.172426	validation_0-t2e:0.183223
[10]	validation_0-rmse:0.163866	validation_0-t2e:0.174084
[11]	validation_0-rmse:0.155743	validation_0-t2e:0.165421
[12]	validation_0-rmse:0.148024	validation_0-t2e:0.157193
[13]	validation_0-rmse:0.140701	validation_0-t2e:0.149393
[14]	validation_0-rmse:0.13374	validation_0-t2e:0.141977
[15]	validation_0-rmse:0.127134	validation_0-t2e:0.134943
[16]	validation_0-rmse:0.120859	validation_0-t2e:0.12827
[17]	validation_0-rmse:0.114901	validation_0-t2e:0.121932
[18]	validation_0-rmse:0.109231	validation_0-t2e:0.115902
[19]	validation_0-rmse:0.103853	validation_0-t2e:0.110187
[20]	validation_0-rmse:0.098752	validation_0-t2e:0.104769
[21]	validation_0-rmse:0.09391	validation_0-t2e:0.099627
[22]	validation_0-rmse:0.089309	validation_0-t2e:0.094736
[23]	validation_0-rmse:0.084936	validation_0-t2e:0.090099
[24]	validation_0-rm

[148]	validation_0-rmse:0.009452	validation_0-t2e:0.009882
[149]	validation_0-rmse:0.00945	validation_0-t2e:0.009879
[150]	validation_0-rmse:0.009448	validation_0-t2e:0.009873
[151]	validation_0-rmse:0.009446	validation_0-t2e:0.00987
[152]	validation_0-rmse:0.009444	validation_0-t2e:0.009869
[153]	validation_0-rmse:0.00944	validation_0-t2e:0.009866
[154]	validation_0-rmse:0.009438	validation_0-t2e:0.009864
[155]	validation_0-rmse:0.009435	validation_0-t2e:0.009861
[156]	validation_0-rmse:0.009431	validation_0-t2e:0.009856
[157]	validation_0-rmse:0.00943	validation_0-t2e:0.009854
[158]	validation_0-rmse:0.009428	validation_0-t2e:0.009852
[159]	validation_0-rmse:0.009426	validation_0-t2e:0.00985
[160]	validation_0-rmse:0.009424	validation_0-t2e:0.009847
[161]	validation_0-rmse:0.009422	validation_0-t2e:0.009844
[162]	validation_0-rmse:0.009421	validation_0-t2e:0.009843
[163]	validation_0-rmse:0.009416	validation_0-t2e:0.009839
[164]	validation_0-rmse:0.009414	validation_0-t2e:0.009836
[1

[40]	validation_0-rmse:0.037525	validation_0-t2e:0.039886
[41]	validation_0-rmse:0.035836	validation_0-t2e:0.038114
[42]	validation_0-rmse:0.034238	validation_0-t2e:0.036448
[43]	validation_0-rmse:0.032724	validation_0-t2e:0.034857
[44]	validation_0-rmse:0.031298	validation_0-t2e:0.033369
[45]	validation_0-rmse:0.029943	validation_0-t2e:0.031959
[46]	validation_0-rmse:0.02867	validation_0-t2e:0.030624
[47]	validation_0-rmse:0.027463	validation_0-t2e:0.029369
[48]	validation_0-rmse:0.026322	validation_0-t2e:0.028173
[49]	validation_0-rmse:0.025242	validation_0-t2e:0.027036
[50]	validation_0-rmse:0.024226	validation_0-t2e:0.02597
[51]	validation_0-rmse:0.023266	validation_0-t2e:0.024962
[52]	validation_0-rmse:0.022362	validation_0-t2e:0.024019
[53]	validation_0-rmse:0.02151	validation_0-t2e:0.023122
[54]	validation_0-rmse:0.020708	validation_0-t2e:0.022287
[55]	validation_0-rmse:0.019954	validation_0-t2e:0.021494
[56]	validation_0-rmse:0.01925	validation_0-t2e:0.02075
[57]	validation_0-r

[181]	validation_0-rmse:0.009231	validation_0-t2e:0.009658
[182]	validation_0-rmse:0.009228	validation_0-t2e:0.009656
[183]	validation_0-rmse:0.009225	validation_0-t2e:0.009653
[184]	validation_0-rmse:0.009224	validation_0-t2e:0.009652
[185]	validation_0-rmse:0.009224	validation_0-t2e:0.009652
[186]	validation_0-rmse:0.009223	validation_0-t2e:0.00965
[187]	validation_0-rmse:0.009223	validation_0-t2e:0.009649
[188]	validation_0-rmse:0.009221	validation_0-t2e:0.009648
[189]	validation_0-rmse:0.009218	validation_0-t2e:0.009645
[190]	validation_0-rmse:0.009217	validation_0-t2e:0.009644
[191]	validation_0-rmse:0.009216	validation_0-t2e:0.009642
[192]	validation_0-rmse:0.009215	validation_0-t2e:0.009642
[193]	validation_0-rmse:0.009214	validation_0-t2e:0.009641
[194]	validation_0-rmse:0.009213	validation_0-t2e:0.00964
[195]	validation_0-rmse:0.009213	validation_0-t2e:0.00964
[196]	validation_0-rmse:0.009212	validation_0-t2e:0.00964
[197]	validation_0-rmse:0.009211	validation_0-t2e:0.009639
[

In [ ]:
X_train, X_val, y_train, y_val = None, None, None, None